

# ME simulations of the replication cost

## Import

In [1]:
from __future__ import division, absolute_import, print_function

import pickle
import random
import math
import pandas as pd
import numpy as np
from scipy.optimize import leastsq

from cobrame.util import building, mu, me_model_interface
from cobrame import mu



## Modified functions from ECOLIme and COBRAme

In [7]:
# Function to solve the model at an specific mu (0.69)

def solve_me_model_fixed(me, muf,using_soplex=False):
    if using_soplex:
        from cobrame.solve.algorithms import solve_at_growth_rate
        solve_at_growth_rate(me, muf)
    else:
        from qminospy.me1 import ME_NLP1
        print('qminos')
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use solv for now
        xopt,status,hs = me_nlp.solvelp(0.69, verbosity=2)
        me.solution.f = me.solution.x_dict['biomass_dilution']

We take some fragments from the [dna_replication.py](https://github.com/SBRG/ecolime/blob/1d7116b32f474dd5adbf9c72e67b22654304469d/ecolime/dna_replication.py) code from the ECOLIme module so that instead of using a preestablished DNA percent we can modify it to increase it.

In [2]:
# Experimental Data
gr_data_doublings_per_hour = [0, 0.6, 1.0, 1.5, 2.0, 2.5]
gr_data = [m * math.log(2) for m in gr_data_doublings_per_hour]

# First point is mass 1 genome in 0.4 um^3 at density 
# Original data that we will be modifying
# percent_dna_data = [0.0592, 0.0512, 0.0330, 0.0252, 0.0222, 0.0208]

def percent_dna_template_function(params, gr):
    [g_p_gdw_0, g_per_gdw_inf, b, d] = params
    c = g_per_gdw_inf
    a = g_p_gdw_0 - g_per_gdw_inf
    g_p_gdw = (-a * gr ** d) / (b + gr ** d) + a + c
    return g_p_gdw


def optimize_dna_function(gr, percent_dna):
    params = np.array([0.9, 0.3, 0.2, 1])

    def _minimization_function(params, gr, percent_dna):
        return percent_dna_template_function(params, gr) - percent_dna
    a = leastsq(_minimization_function, params, args=(gr, percent_dna))
    return a[0]


def get_dna_mw_no_ppi_dict(model):
    """
    Return the molecular weight of dna component with the diphosphate group
    removed.
    """
    dna_mw_no_ppi = {}
    ppi_mw = model.metabolites.ppi_c.formula_weight
    for dna in ['dctp', 'dgtp', 'datp', 'dttp']:
        dna_mw = model.metabolites.get_by_id(dna + '_c').formula_weight
        dna_mw_no_ppi[dna] = dna_mw - ppi_mw

    return dna_mw_no_ppi



# We modify this function to instead of taking a global percent_dna_data paramaeter
# now it need a local percent_dna_data
def return_gr_dependent_dna_demand(model, gc_fraction, percent_dna_data):
    """
    Returns dNTP coefficients and lower/upper bounds of DNA_replication
    reaction
    """

    fit_params = optimize_dna_function(gr_data, percent_dna_data)
    dna_g_per_g = percent_dna_template_function(fit_params, mu)  # gDNA / gDW

    # average dinucleotide molecular weight
    dna_mw = get_dna_mw_no_ppi_dict(model)
    dntp_mw = (gc_fraction * (dna_mw['dctp'] + dna_mw['dgtp'])) / 2 + \
              ((1 - gc_fraction) * (dna_mw['datp'] + dna_mw['dttp'])) / 2

    # 1 / (gDNA / mol) * (1000 mmol / 1 mol)
    mmol_dntps_per_gram_dna = 1 / dntp_mw * 1000

    # (mmol / gDNA) * (gDNA / gDW)
    mmol_dntp_per_gdw = dna_g_per_g * mmol_dntps_per_gram_dna

    # lower and upper bound
    mmol_dntp_per_gdw_per_hr = mmol_dntp_per_gdw * mu

    # Fraction of each nucleotide in DNA, based on gc_fraction
    dna_demand_stoich = {
        'datp_c': -((1 - gc_fraction) / 2),
        'dctp_c': -(gc_fraction / 2),
        'dgtp_c': -(gc_fraction / 2),
        'dttp_c': -((1 - gc_fraction) / 2),
        'ppi_c': 1}

    return dna_demand_stoich, mmol_dntp_per_gdw_per_hr

## Simulations

We will be modifying the percent DNA data by multiplying it by 1, 1.125, 1.25 and 1.5



In [8]:
valores = [1, 1.125, 1.25, 1.5]
modelos = {}
nombres = [str(n) for n in valores]


for n,valor in enumerate(valores):
    
    # Original DNA percentage
    ## First point is mass 1 genome in 0.4 um^3 at density 
    percent_dna_data = [0.0592, 0.0512, 0.0330, 0.0252, 0.0222, 0.0208]
    
    # Modify the values
    percent_dna_data = [dna*valor for dna in percent_dna_data]
    
    # Open original model
    with open('../../files/models/iJL1678b.pickle', 'rb') as f:
        modelito = pickle.load(f)
    
    # Make new calculations withthe new DNA percentages
    dna_demand_stoich, dna_demand_bound = return_gr_dependent_dna_demand(
            modelito, modelito.global_info['GC_fraction'], percent_dna_data)
    
    # Constrain the model to the new DNA demand 
    modelito.reactions.get_by_id('DNA_replication').upper_bound = dna_demand_bound
    modelito.reactions.get_by_id('DNA_replication').lower_bound = dna_demand_bound

    solve_me_model_fixed(modelito,0.69)
    modelos[nombres[n]] = modelito



qminos
Finished compiling expressions in 168.941200 seconds
Finished substituting S,lb,ub in 3.673695 seconds
Finished makeME_LP in 0.702486 seconds
Getting MINOS parameters from ME_NLP...
qminos
Finished compiling expressions in 166.359216 seconds
Finished substituting S,lb,ub in 3.910021 seconds
Finished makeME_LP in 0.762523 seconds
Getting MINOS parameters from ME_NLP...
qminos
Finished compiling expressions in 168.575055 seconds
Finished substituting S,lb,ub in 3.347258 seconds
Finished makeME_LP in 0.594482 seconds
Getting MINOS parameters from ME_NLP...
qminos
Finished compiling expressions in 163.087296 seconds
Finished substituting S,lb,ub in 3.625209 seconds
Finished makeME_LP in 0.646928 seconds
Getting MINOS parameters from ME_NLP...


## Save the models for analysis

In [9]:
for modelo in modelos:
    with open('../../files/models/DNA_per'+modelo+'.pickle', 'wb') as f:
        pickle.dump(modelos[modelo], f)